In [1]:
from src.data_structures import Instance
import torch

In [2]:
size = 5000
instance = Instance.generate(size,20,seed=200) #This has to be faster

In [3]:
#Precalc

synSet = instance.synSet
gains = torch.tensor(list(instance.polynomial_gains.values()))
costs = instance.costs.T
budget = instance.budget
gamma = instance.gamma
profits = torch.tensor(instance.profits)
lower_costs = torch.tensor(costs[0])
upper_costs = torch.tensor(costs[1])
delta_costs = upper_costs-lower_costs

In [4]:
#Yes, this is bad, but im lazy
sol = torch.zeros(size,dtype=torch.int64)
for i in range(size):
    if torch.rand(1).item() > 0.7:
        sol[i] = 1

a = instance.evaluate(sol)
print(sol,a)


tensor([0, 1, 0,  ..., 0, 0, 0]) 70683.95470246754


In [5]:
for i in range(500):
    a = instance.evaluate(sol)

In [6]:
def a():
    item_vec = torch.arange(1,size+1)
    indices_ordenados = torch.argsort(delta_costs,descending=True) #Ordenar los indices primero en base a los delta_costs, me ahorra sorts
    item_vec = item_vec[indices_ordenados]
    ordered_sol = sol[indices_ordenados]
    investments = torch.multiply(ordered_sol,item_vec)
    investments = investments[investments > 0] - 1
    total_upper_costs = torch.sum(upper_costs[investments[:gamma]])
    total_lower_costs = torch.sum(lower_costs[investments[gamma:]])
    total_costs = total_upper_costs + total_lower_costs
    if total_costs <= budget:
        of = torch.sum(profits[investments]) - total_costs
        investment_set = set(investments)
        for i,syn in enumerate(synSet):
            #es_subconjunto = torch.isin(torch.tensor(list(syn)), investments).all().item()
            if syn.issubset(investment_set):
                of += gains[i]

    return of

In [7]:
for i in range(500):
    b = a()

In [8]:
import torch

# Definir los tensores
tensor_grande = torch.tensor([1, 2, 3, 4])
tensor_pequeno = torch.tensor([[3,2],[5,6]])

# Verificar si tensor_pequeno es un subconjunto de tensor_grande
es_subconjunto = torch.isin(tensor_pequeno, tensor_grande)
print(es_subconjunto)

tensor([[ True,  True],
        [False, False]])
